In [1]:
import pandas as pd
import numpy as np
import os
import nltk
import re
from datetime import date

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense ,Dropout,LSTM, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.losses import sparse_categorical_crossentropy

sparse_categorical_crossentropy

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

<function tensorflow.python.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)>

In [2]:
badlist = pd.read_csv('data/badlist_de', header=None)[0].tolist()

In [3]:
def clean_line(line):
    
    for badchar in badlist:
        if badchar in line:
            line = re.sub(badchar,'',line)
    
    return line

In [4]:
# read the raw text files and clean the lines

all_episodes_by_sentence = []

seperators =  [' \t\n', '\t\n']

with open('Newssnippets.txt',encoding='utf-8' ) as in_raw:
    # start token
    for (i, line) in enumerate(in_raw):
        if not line in seperators:
            all_episodes_by_sentence.append(clean_line(line) )
        

In [5]:
all_episodes_by_sentence[0]

'Donnerstag 08.08.2019 17:07 - Übermedien\n'

In [6]:
# combine text to create a single string for sliceshifting
text = ' '.join(all_episodes_by_sentence)

In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

105 unique characters


In [17]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [18]:
text_as_int

array([31, 70, 69, ..., 60, 73,  0])

In [19]:
# The maximum length sentence we want for a single input in characters
seq_length = 150
examples_per_epoch = len(text)//(seq_length+1)

In [20]:
text_as_int

array([31, 70, 69, ..., 60, 73,  0])

In [21]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [22]:
# extract sequences from character dataset
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [23]:

def split_shift_input(segment):
    """
    Creates the teaching data by shifting the training data on off to create labeled data
    """
    input_segment = segment[:-1]
    target_segment = segment[1:]
    return input_segment, target_segment

dataset = sequences.map(split_shift_input)

In [24]:
dataset

<MapDataset shapes: ((150,), (150,)), types: (tf.int64, tf.int64)>

In [25]:
# set up dataset as prebatched
BATCH_SIZE = 35

# Length of the vocabulary
vocab_size = len(vocab)

# embedding dimension
embedding_dim = 150

# RNN units
rnn_units = 256

dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [26]:
dataset

<BatchDataset shapes: ((35, 150), (35, 150)), types: (tf.int64, tf.int64)>

In [27]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    
    i = Input(shape=(None,), batch_size=batch_size )
    x = Embedding(vocab_size, embedding_dim)(i)
    x = LSTM(rnn_units, 
             return_sequences=True,
             stateful=True,
             recurrent_initializer='glorot_uniform')(x)
    x = Dense(vocab_size)(x)
    x = Dense(vocab_size)(x)

    model = Model(i,x)
    
    return model
    

In [28]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [29]:
# simple model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(35, None)]              0         
_________________________________________________________________
embedding (Embedding)        (35, None, 150)           15750     
_________________________________________________________________
unified_lstm (UnifiedLSTM)   (35, None, 256)           416768    
_________________________________________________________________
dense (Dense)                (35, None, 105)           26985     
_________________________________________________________________
dense_1 (Dense)              (35, None, 105)           11130     
Total params: 470,633
Trainable params: 470,633
Non-trainable params: 0
_________________________________________________________________


In [30]:
def loss(labels, logits):
    """
    Define loss function 
    """
    return sparse_categorical_crossentropy(labels, logits, from_logits=True)


In [31]:
model.compile(optimizer='adam', loss =loss)# loss='sparse_categorical_crossentropy' )

In [32]:
# Directory where the checkpoints will be saved
today = date.today()

checkpoint_dir = './base_training_checkpoints_{today}'.format(today=today)


checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

# define callbacks
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [33]:
EPOCHS=50

In [34]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
265/265 [==============================] - 5s 18ms/step - loss: 2.7759
Epoch 2/50
265/265 [==============================] - 3s 12ms/step - loss: 2.1733
Epoch 3/50
265/265 [==============================] - 3s 12ms/step - loss: 1.9777
Epoch 4/50
265/265 [==============================] - 3s 12ms/step - loss: 1.8540
Epoch 5/50
265/265 [==============================] - 3s 12ms/step - loss: 1.7603
Epoch 6/50
265/265 [==============================] - 3s 12ms/step - loss: 1.6852
Epoch 7/50
265/265 [==============================] - 3s 12ms/step - loss: 1.6239
Epoch 8/50
265/265 [==============================] - 3s 12ms/step - loss: 1.5730
Epoch 9/50
265/265 [==============================] - 3s 12ms/step - loss: 1.5304
Epoch 10/50
265/265 [==============================] - 3s 12ms/step - loss: 1.4941
Epoch 11/50
265/265 [==============================] - 3s 12ms/step - loss: 1.4631
Epoch 12/50
265/265 [==============================] - 3s 12ms/step - loss: 1.4361
Epoch 13/50
2

In [35]:
# for prediction, batch size has to be changed
# So reload the model and set shape to [1, None]

# preloaded one checkpoint directory

tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(1, None)]               0         
_________________________________________________________________
embedding_1 (Embedding)      (1, None, 150)            15750     
_________________________________________________________________
unified_lstm_1 (UnifiedLSTM) (1, None, 256)            416768    
_________________________________________________________________
dense_2 (Dense)              (1, None, 105)            26985     
_________________________________________________________________
dense_3 (Dense)              (1, None, 105)            11130     
Total params: 470,633
Trainable params: 470,633
Non-trainable params: 0
_________________________________________________________________


In [37]:
def text_gen(model, start_string, freedom=1.0, num_generate=1000):
    """
    generate text with the trained model
    
    start_string (STR):  Basis for the model to start prediction on. 
    freedom (FLOAT): Multiplier for predictions. The lower it is the lower the impact of predictive variance
    num_generate (INT): Desired text length
    """
    
    text_generated = []
    

    # vectorization of starting string
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    model.reset_states()
    
    
    for i in range(num_generate):
        predictions = model(input_eval)
        
        
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / freedom
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [43]:
print(text_gen(model, start_string=u"Montag", freedom=0.5, num_generate=500))

Montagg-·n-SQüstenruppen und dem Swassen und in der Sänger eine Ex-Freundin Laura Müller (19) scheinen Sänger Werber Lars Steiling musste er in der Band am Millionen Rammstein und der Sänger erscheinen Sieg geht die Sänger einen der Unterhaltung von George Rosenhe als Sänger Ben Affleck (42) haben die Trikot-Star und Deutschen Trico von Sänger Things die Ehe randem dazu: Er lustig an den Sänger und dessen Sohn Tom Kaulitz (30) zur Verfüg in der Samuel die Sänger für den Sänger und weitere Produktionsf
